# PyTorch基础

## 1 Tensor 张量

In [1]:
import torch

a = torch.Tensor([[2, 3], [4, 8], [7, 9]])  # 默认是FloatTensor类型
print(f'a: {a}')
print(f'a.size(): {a.size()}')

a: tensor([[2., 3.],
        [4., 8.],
        [7., 9.]])
a.size(): torch.Size([3, 2])


In [2]:
torch.LongTensor([[2, 3], [4, 8], [7, 9]])  # 指定类型

tensor([[2, 3],
        [4, 8],
        [7, 9]])

In [3]:
torch.zeros((3, 2))

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [4]:
torch.randn((3, 2))  # 正态分布随机数

tensor([[ 0.9545, -0.2745],
        [ 1.5934, -0.1374],
        [ 0.3876, -1.0731]])

In [5]:
a[0, 1] = 100  # 修改元素值
a

tensor([[  2., 100.],
        [  4.,   8.],
        [  7.,   9.]])

In [6]:
a.numpy()  # 转numpy.array类型

array([[  2., 100.],
       [  4.,   8.],
       [  7.,   9.]], dtype=float32)

In [7]:
import numpy as np
e = np.array([[2,3], [4,5]])
f = torch.from_numpy(e)  # array转torch张量
f

tensor([[2, 3],
        [4, 5]], dtype=torch.int32)

In [8]:
f.float()  # 转成 FloatTensor

tensor([[2., 3.],
        [4., 5.]])

In [9]:
torch.cuda.is_available()  # 是否可以调用GPU

True

In [10]:
g = a.cuda()  # 把变量a转成cuda数据类型放大gpu上，需要定义一个新变量存储
g

tensor([[  2., 100.],
        [  4.,   8.],
        [  7.,   9.]], device='cuda:0')

## 2 Variable / autograd

In [11]:
from torch.autograd import Variable

# 输出值只有一个的aotograd
w = Variable(torch.Tensor([2, 3, 4]).reshape((1,3)), requires_grad=True)
x = Variable(torch.Tensor([1, 2, 3]).reshape((3,1)), requires_grad=True)
b = Variable(torch.Tensor([5]), requires_grad=True)
y = torch.matmul(w, x) + b
print(' y=', y)

y.backward()
print('dw=', w.grad)
print('dx=', x.grad)

 y= tensor([[25.]], grad_fn=<AddBackward0>)
dw= tensor([[1., 2., 3.]])
dx= tensor([[2.],
        [3.],
        [4.]])


In [12]:
# 多输出值：可以求平均值后autograd
w.grad.zero_()
x = Variable(torch.Tensor([[1, 2], [3, 4], [5, 6]]), requires_grad=True)  # 3*2
y = torch.mean(torch.matmul(w, x) + b)
print(' y=', y)

y.backward()
print('dw=', w.grad)
print('dx=', x.grad)

 y= tensor(40.5000, grad_fn=<MeanBackward0>)
dw= tensor([[1.5000, 3.5000, 5.5000]])
dx= tensor([[1.0000, 1.0000],
        [1.5000, 1.5000],
        [2.0000, 2.0000]])


In [13]:
# 多输出值：也可以输出的每个维度单独求梯度，最后求和
w.grad.zero_()  # 清除已有的梯度值
x.grad.zero_()
y = torch.matmul(w, x) + b  # 1*2
print(' y=', y)

# 因为y的值不只一个，需要每个独立推回去后叠加；这里也可以不用ones，自定义每个维度权重
y.backward(torch.ones_like(y))
print('dw=', w.grad)
print('dx=', x.grad)

 y= tensor([[36., 45.]], grad_fn=<AddBackward0>)
dw= tensor([[ 3.,  7., 11.]])
dx= tensor([[2., 2.],
        [3., 3.],
        [4., 4.]])


# 神经网络

In [14]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def rose_points(m, nclass=2, diameter=4, disturbance=0.2, seed=None):
    """ 玫瑰图的坐标数据（一般用于造测试数据）
    :param m: 点数量，样本数
    :param nclass: 类别数，默认二分类nclass=2
    :class_petals: （该参数废弃，因为并不能要多少给多少，有个限制，所以不妨限制死，每种4个花瓣）每一类的花瓣数
    :param diameter: 极径长度
    :param disturbance: 随机扰动的程度
    :param seed: 随机数种子
    :return: 两个np.array矩阵 X, Y
        X.shape -> (m, 2)，每一行是每个点的横纵坐标
        Y.shape -> (m, 1)，用0、1、2...的值标记类别
        
    ref: https://en.wikipedia.org/wiki/Rose_(mathematics)
    """
    from math import pi
    # 1 参数计算
    if seed: np.random.seed(seed)
    n = int(m / nclass) # 每一类的点的个数
    X = np.zeros((m, 2))
    Y = np.zeros((m, 1), dtype='uint8') # label 向量，0 表示红色，1 表示蓝色
    
    alpha = 2 * pi / nclass  # 每一个花瓣的旋转角度
    
    # 2 生成点
    for j in range(nclass):
        ix = range(n*j, n*(j+1))
        t = np.linspace(j*alpha, (j+1)*alpha, n) + np.random.randn(n)*disturbance  # theta
        r = diameter*np.sin(2*nclass*t) + np.random.randn(n)*disturbance  # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j
        
    return X, Y

In [ ]:
# test: rose_points
X, Y = rose_points(400, 3, disturbance=0, seed=1)
plt.scatter(X[:, 0], X[:, 1], c=Y[:, 0], cmap=plt.cm.Spectral)  # https://matplotlib.org/tutorials/colors/colormaps.html?highlight=colormap

In [ ]:
def plot_decision_boundary(model, x, y):
    """ 可视化决策边界
    :param model: 预测X值的模型，输入x，返回(m*1)的yhat
    :param x: (m样本*n特征)的x
    :param y: (m*1)的实际标签
    """
    # Set min and max values and give it some padding
    x_min, x_max = x[:, 0].min() - 1, x[:, 0].max() + 1
    y_min, y_max = x[:, 1].min() - 1, x[:, 1].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(x[:, 0], x[:, 1], c=y.reshape(-1), s=40, cmap=plt.cm.Spectral)

In [ ]:
import time

import torch
from torch import nn
from torch.autograd import Variable


class module_net(nn.Module):
    def __init__(self, num_input, num_hidden, num_output):
        """
        TODO 这个层数设置应该可以写的更灵活一些的吧~~
        """
        super(module_net, self).__init__()
        self.layer1 = nn.Linear(num_input, num_hidden)
        self.layer2 = nn.Tanh()
        self.layer3 = nn.Linear(num_hidden, num_output)
        
    def forward(self, x):
        if isinstance(x, np.ndarray):
            x = torch.from_numpy(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

    def train(self, x, y, times, print_interval=None):
        """
        TODO 有些训练方法是独立于模型的，获取应该封装在类外
        TODO nn.Module本来就有train，最好不要同名覆盖
        """
        # 定义优化器
        optim = torch.optim.SGD(mo_net.parameters(), 1.)
        # 定义损失函数
        criterion = nn.BCEWithLogitsLoss()
        # 训练
        for e in range(times):
            out = self.forward(Variable(x))
            loss = criterion(out, Variable(y))
            optim.zero_grad()
            loss.backward()
            optim.step()
            if print_interval and (e + 1) % print_interval == 0:
                print('epoch: {}, loss: {}'.format(e+1, loss.data))

                
start = time.time()
mo_net = module_net(2, 4, 1)
# 数据
x, y = rose_points(400, seed=1)
mo_net.train(torch.Tensor(x), torch.Tensor(y), 1000, 100)
print(f'During Time: {time.time() - start:.3f} s')